In [1]:
import mediapipe as mp
import cv2
import numpy as np
import autopy
import math
########
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume
########

In [2]:
########
devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(
    IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = cast(interface, POINTER(IAudioEndpointVolume))
vRange = volume.GetVolumeRange()
minVol = vRange[0]
maxVol = vRange[1]
########

#Enter the width and height of your webcam
wCam, hCam = 640, 480
cap = cv2.VideoCapture(0)

mpHands = mp.solutions.hands
hands = mpHands.Hands()
npdraw = mp.solutions.drawing_utils

frameR = 50
hFrameR = 100
smoothening = 3
plocX, plocY = 0, 0 #previous location
clocX, clocY = 0, 0 #current location

wScr, hScr = autopy.screen.size()

while True:
    fingers=[0,0,0,0]
    success, img = cap.read()
    h, w, c = img.shape
    black = np.zeros((img.shape[0], img.shape[1], img.shape[2]))
    RGBimg = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    results = hands.process(RGBimg) #process the frame and give results
    if results.multi_hand_landmarks :
        lmList = []
        for handslms in results.multi_hand_landmarks :
            for ids, lm in enumerate(handslms.landmark):
                cx, cy = int(lm.x * w), int(lm.y * h)
                lmList.append([ids,cx,cy])
            
        for handlms in results.multi_hand_landmarks:
            npdraw.draw_landmarks (black, handlms, mpHands.HAND_CONNECTIONS)
        
        for i in range(0,4):
            if lmList[8+i*4][2] < lmList[(8+i*4)-2][2]: # finger is open
                fingers[i]=1
                
        if (fingers[0]==1 and fingers[1]==0 and fingers[2]==0 and fingers[3]==0):
#             print("Mouse")
            cv2.rectangle(black, (frameR,frameR),(wCam-frameR,hCam-hFrameR) , (100,100,100) , 1)
            x1, y1 = lmList[8][1:]
            #Convert Coordinates
            x3 = np.interp(x1, (frameR, wCam - frameR), (0, wScr))
            y3 = np.interp(y1, (frameR, hCam - hFrameR), (0, hScr))
            clocX = plocX + (x3 - plocX) / smoothening
            clocY = plocY + (y3 - plocY) / smoothening
            autopy.mouse.move(wScr - clocX, clocY) # wScr-clocX for inverting the horizontal movement
            print(clocX,clocY)
            plocX, plocY = clocX, clocY
        
        if (fingers[0]==1 and fingers[1]==1 and fingers[2]==0 and fingers[3]==0):
            if not(last_state_fingers == fingers):
                autopy.mouse.click()
        
        if (fingers[0]==1 and fingers[1]==0 and fingers[2]==0 and fingers[3]==1):
            print("Volume")
            x1, y1 = lmList[8][1:]
            x2, y2 = lmList[20][1:]
            cx, cy = (x1 +x2)//2 , (y1 +y2)//2
            cv2.line(black, (x1,y1),(x2,y2),(0,255,0),3)
            cv2.circle(black,(cx,cy),10, (100,100,100),-1)
            dist = math.dist((x1,y1),(x2,y2))
            print(dist)
            #Dist range 40-180 and volume range conversion
            vol = np.interp(dist,[40,180],[minVol,maxVol])
            volume.SetMasterVolumeLevel(vol, None)
        last_state_fingers = fingers
    cv2.imshow("black image",black)  
    k=cv2.waitKey(1)
    if k == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

237.9851851851852 65.45454545454545
400.4345679012346 113.45454545454545
510.6304526748971 144.5818181818182
587.8869684499314 166.20606060606062
640.3394604481024 182.36767676767678
672.4633440024386 193.14208754208755
691.0348219275517 199.45230078563412
699.6232146183678 204.53183688739244
704.4006615974304 207.91819428856465
707.5856262501388 211.0484931620734
702.1237508334259 212.26263180501863
681.4158338889506 208.70842120334575
658.1290744444856 204.59349292344262
631.2267903703978 199.23202558532537
604.7586009876726 193.912259481126
578.5798080658558 187.74756692681126
559.230983155015 182.76504461787417
570.0354702514915 178.57063580585552
605.6829060935869 178.39254508269156
649.3589744327617 182.63745429755195
702.1800570292486 192.44921195594372
744.9792972787583 204.22674736456855
782.9936055932462 221.67843763698508
808.3364778029049 233.3128978185961
825.2317259426774 242.81465915179135
837.4433728506738 252.64007579816393
844.63632264119 265.29944447150325
849.431622

675.2521577521211 515.7643059709277
680.5681051680807 513.151961556376
695.4898478898316 564.6467622497053
632.4302689635914 438.3948111967732
597.9757348646165 352.4813892826973
577.8504899097443 296.07850194604066
563.4855117916813 261.96748614584527
553.9088597129728 240.97226349116957
555.1096101790189 233.08453929714335
568.2360364156423 239.1715110465804
584.572172425243 253.70221948559904
595.4629297649768 267.7529948085812
618.8419531766513 298.0656329026905
659.0798206362861 349.6922401169452
688.7495100538204 395.45543280523617
707.581154850695 426.8369552034908
722.0318810115745 457.35797013566054
729.7694021558644 500.3962225146828
748.2018236594652 523.8520271310007
762.386400958162 548.2165029358187
785.1168598980339 576.6776686238791
790.7890177098004 598.2699609007678
786.9852710657929 602.1920951459664
778.760551080899 603.9341240367048
768.5366636835623 606.840931175985
760.7725906038564 607.9060753294445
753.7002455877562 609.4888987044782
747.0890526140597 610.54411

490.9961470744026 343.4403794574947
496.10113508663875 386.05116206257225
498.5563122799814 417.94925955686637
495.4523563348024 438.34193061366847
